<a href="https://colab.research.google.com/github/MehrdadJannesar/CNN_AdaBoost/blob/master/ResNet101_laitec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import libraries
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense,Input
from keras.applications.resnet import ResNet101
import numpy as np

Using TensorFlow backend.


In [2]:
#Load Dataset
#Optimize and categorize dataset
(train_images,train_labels),(test_images,test_labels) = cifar10.load_data()

X_train = train_images.astype('float32')
X_test = test_images.astype('float32')

X_train /=255
X_test /=255
N_classes = 10
Y_train = keras.utils.to_categorical(train_labels,N_classes)
Y_test = keras.utils.to_categorical(test_labels,N_classes)

170500096/170498071 [==============================] - 11s 0us/step


In [3]:
#Create model
"""
---> input layer 
---> close Resnet50 layers
---> fullyConnect 10 Classes

I don't open ResNet layer in this code, But I explained it in previous code (ResNet50_laitec)
"""
input_layer = Input(shape=(32,32,3))
resnet = ResNet101(include_top = False, pooling = 'avg',weights=None)(input_layer)
dense = Dense(10,activation='softmax')(resnet)

mymodel = Model(input_layer, dense)

In [4]:
#Compile my model and Defining optimizers
#Optimizer is SGD (Stochastic Gradient Descent) and loss function is Crossentropy
sgd = keras.optimizers.SGD(lr=0.01, decay = 1e-6,momentum=0.9,nesterov=True)
mymodel.compile(optimizer=sgd ,loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [14]:
#Fit myModel
net = mymodel.fit(X_train,Y_train, batch_size=128,epochs=10)

Epoch 1/10
50000/50000 [==============================] - 61s 1ms/step - loss: 0.5366 - acc: 0.8347
Epoch 2/10
50000/50000 [==============================] - 61s 1ms/step - loss: 0.5030 - acc: 0.8462
Epoch 3/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.4799 - acc: 0.8572
Epoch 4/10
50000/50000 [==============================] - 61s 1ms/step - loss: 0.5689 - acc: 0.8328
Epoch 5/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.4773 - acc: 0.8635
Epoch 6/10
50000/50000 [==============================] - 61s 1ms/step - loss: 0.4424 - acc: 0.8746
Epoch 7/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.3986 - acc: 0.8895
Epoch 8/10
50000/50000 [==============================] - 63s 1ms/step - loss: 0.3675 - acc: 0.9001
Epoch 9/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.3494 - acc: 0.9071
Epoch 10/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.3338 - acc: 0.9131

In [15]:
#Predict network
#Validation myModel (Loss,Accurcy)
test_loss, test_acc = mymodel.evaluate(X_test, Y_test)
test_prediction = mymodel.predict(X_test)


print("test_loss: ",test_loss)
print("test_acc: ",test_acc)
print("test_prediction", test_prediction)

10000/10000 [==============================] - 9s 907us/step
test_loss:  2.1956620325088503
test_acc:  0.5551
test_prediction [[7.8654237e-05 6.0090120e-04 5.6901299e-03 ... 7.6884913e-05
  1.9280907e-03 1.3088972e-03]
 [8.8981236e-05 8.2478749e-05 5.2503552e-08 ... 1.3602178e-12
  9.9534494e-01 4.4835052e-03]
 [5.4917741e-04 9.8002481e-04 1.3027161e-06 ... 6.5098597e-08
  9.5274866e-01 4.5719810e-02]
 ...
 [1.8268431e-02 4.8153140e-03 1.7244960e-01 ... 7.4495472e-02
  1.1325207e-03 2.1502201e-03]
 [3.2949891e-05 1.0229479e-02 8.2259365e-05 ... 3.2498199e-02
  2.2982243e-03 2.7225579e-03]
 [9.0687972e-06 1.8935613e-13 4.0436455e-05 ... 9.9994671e-01
  2.9837344e-12 1.1680923e-09]]


ResNet
Deep residual networks were a breakthrough idea which enabled the development of much deeper networks (hundreds of layers as opposed to tens of layers).

Its a generally accepted principle that deeper networks are capable of learning more complex functions and representations of the input which should lead to better performance. However, many researchers observed that adding more layers eventually had a negative effect on the final performance. This behavior was not intuitively expected, as explained by the authors below.

Let us consider a shallower architecture and its deeper counterpart that adds more layers onto it. There exists a solution by construction to the deeper model: the added layers are identity mapping, and the other layers are copied from the learned shallower model. The existence of this constructed solution indicates that a deeper model should produce no higher training error than its shallower counterpart. But experiments show that our current solvers on hand are unable to find solutions that are comparably good or better than the constructed solution (or unable to do so in feasible time).

This phenomenon is referred to by the authors as the degradation problem - alluding to the fact that although better parameter initialization techniques and batch normalization allow for deeper networks to converge, they often converge at a higher error rate than their shallower counterparts. In the limit, simply stacking more layers degrades the model's ultimate performance.

The authors propose a remedy to this degradation problem by introducing residual blocks in which intermediate layers of a block learn a residual function with reference to the block input. You can think of this residual function as a refinement step in which we learn how to adjust the input feature map for higher quality features. This compares with a "plain" network in which each layer is expected to learn new and distinct feature maps. In the event that no refinement is needed, the intermediate layers can learn to gradually adjust their weights toward zero such that the residual block represents an identity function.

![](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-6.29.19-PM.png)

Note: It was later discovered that a slight modification to the original proposed unit offers better performance by more efficiently allowing gradients to propagate through the network during training.

![](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-17-at-10.36.21-PM.png)

Wide residual networks
Although the original ResNet paper focused on creating a network architecture to enable deeper structures by alleviating the degradation problem, other researchers have since pointed out that increasing the network's width (channel depth) can be a more efficient way of expanding the overall capacity of the network.

Architecture

![](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-6.30.05-PM.png)

Each colored block of layers represent a series of convolutions of the same dimension. The feature mapping is periodically downsampled by strided convolution accompanied by an increase in channel depth to preserve the time complexity per layer. Dotted lines denote residual connections in which we project the input via a 1x1 convolution to match the dimensions of the new block.

The diagram above visualizes the ResNet 34 architecture. For the ResNet 50 model, we simply replace each two layer residual block with a three layer bottleneck block which uses 1x1 convolutions to reduce and subsequently restore the channel depth, allowing for a reduced computational load when calculating the 3x3 convolution.

![alt text](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-5.47.38-PM.png)

Parameters: 25 million (ResNet 50)

Papers:

Deep Residual Learning for Image Recognition
Identity Mappings in Deep Residual Networks
Wide Residual Networks

ResNet-101:

![alt text](https://resources.wolframcloud.com/NeuralNetRepository/resources/images/853/8536f93a-af9a-4f71-a516-87eee51cc193-io-1-o.en.gif)